# Programming Assignment 2: Naïve Bayes
## Part 2: Classification

#### Name: Umama Nasir Abbasi
#### Roll Number: 23100265

### Instructions
*   In this part of the assignment you will be implementing a Naïve Bayes Classification Model for classification from scratch.
*   Your code must be in the Python programming language.
*   You are encouraged to use procedural programming and throughly comment your code.
*   For Part 2, you can only use standard libraries i.e. numpy, pandas, regex, matplotlib and scipy. You are **not** allowed to use any machine learning toolkits or libraries for training and testing your model. 
*   **Carefully read the submission guidelines, plagiarism and late days policy.**

### Submission Guidelines
Submit your code both as notebook file (.ipynb) and python script (.py) as individual files on LMS. Name both files as RollNumber_PA2_PartNum, i.e. this part should be named as `2xxxxxxx_PA4_2`. If you don’t know how to save .ipynb as .py see [this](https://i.stack.imgur.com/L1rQH.png). Failing to submit any one of them might result in the reduction of marks. All cells **MUST** be run to get credit.

### Plagiarism Policy
The code **MUST** be done independently. Any plagiarism or cheating of work from others or the internet will be immediately referred to the DC. If you are confused about what constitutes plagiarism, it is **YOUR** responsibility to consult with the instructor or the TA in a timely manner. No “after the fact” negotiations will be possible. The only way to guarantee that you do not lose marks is **DO NOT LOOK AT ANYONE ELSE'S CODE NOR DISCUSS IT WITH THEM**.

### Late Days Policy

The deadline for the assignment is final. However, in order to accommodate all the 11th-hour issues, there is a late submission policy i.e. you can submit your assignment within 3 days after the deadline with a 25% deduction each day.


### Introduction

In this part, you will be implementing and training a Naïve Bayes Classification model to classify news articles into different categories. Specifically, you will need to implement [this](https://drive.google.com/file/d/1lNXwNpU6ydeP8I6cPsXLdbA4RQu17G9E/view?usp=sharing) algorithm. For more details of the algorithm and insights into the Naïve Bayes, you can also consult [Chapter 4](https://web.stanford.edu/~jurafsky/slp3/4.pdf) of the Speech and Language Processing book as reference.


### Dataset
You will be using the [AG News Classification Dataset](https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset) for the purposes of this part of the assignment. This contains 120,000 training samples and 7600 testing samples of news articles belonging to 4 categories: World, Sports, Business, and Sci/Tech. 
The data has already been spilt into train and test files.
You're required to implement a model that classifies the articles among the 4 aforementioned categories.

Start by importing all required libraries here.

In [1]:
import re
import numpy as np
import pandas as pd
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2.1 - Loading and Preprocessing the Dataset

Load the dataset - both training and test samples

In [3]:
# code here
test = pd.read_csv("/content/drive/MyDrive/Fall2022-2023/CS535ML/PA4/DataP2/test.csv")
train = pd.read_csv("/content/drive/MyDrive/Fall2022-2023/CS535ML/PA4/DataP2/train.csv")

In [4]:
train.head(5)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


Use regex to pre-process the data loaded. Go through the data and use your own discretion to decide on what kind of pre-processing might be required.

In [5]:
# code here
file=open('/content/drive/MyDrive/Fall2022-2023/CS535ML/PA4/DataP2/stop_words.txt','r')
stopWords = file.read()
punct =  r'[^ \\t\d\w\.]'
digits = r'[0-9]'
fullStop = r'\.'
slash = r'\\'
underscore = r'\_'

trainCleaned = train.applymap(lambda s:s.lower() if type(s) == str else s)
trainCleaned = trainCleaned.replace(to_replace = punct, value = '', regex = True)
trainCleaned = trainCleaned.replace(to_replace = fullStop, value = '', regex = True)
trainCleaned = trainCleaned.replace(to_replace = underscore, value = ' ', regex = True)
trainCleaned = trainCleaned.replace(to_replace = slash, value = '', regex = True)
trainCleaned = trainCleaned.replace(to_replace = stopWords, value = '', regex = True)
trainCleaned = trainCleaned.replace(to_replace = digits, value = '', regex = True)
trainCleaned['info'] = trainCleaned['Title'] + " " + trainCleaned['Description']
trainCleaned = trainCleaned.drop(["Title","Description" ], axis=1)


In [6]:
test = test.applymap(lambda s:s.lower() if type(s) == str else s)
test = test.replace(to_replace = punct, value = '', regex = True)
test = test.replace(to_replace = fullStop, value = '', regex = True)
test = test.replace(to_replace = underscore, value = ' ', regex = True)
test = test.replace(to_replace = slash, value = '', regex = True)
test = test.replace(to_replace = stopWords, value = '', regex = True)
test = test.replace(to_replace = digits, value = '', regex = True)
test['info'] = test['Title'] + " " + test['Description']
test = test.drop(["Title","Description"], axis=1)


Show the results of your preprocessing by printing a random sample of 10 articles before and after pre-processing.

In [7]:
# code here
print(trainCleaned.sample(n=10, random_state=42, replace=False))
print(train.sample(n=10, random_state=42, replace=False))

        Class Index                                               info
71787             3  bbc set for major shakeup claims newspaper lon...
67218             3  marsh averts cash crunch embattled insurance b...
54066             2  jeter yankees look to take control ap ap  dere...
7168              4  flying the sun to safety when the genesis caps...
29618             3  stocks seen flat as nortel and oil weigh  new ...
101425            2  inter milan seeks redemption win against juven...
20441             3  saudi arabia cuts oil prices oil prices eased ...
2662              1  google cuts its ipo price range san jose calif...
20371             3  focus santander says hbos counterbid to face p...
108151            4  hp revises cluster plans hp quote chart is dro...
        Class Index                                              Title  \
71787             3       BBC set for major shake-up, claims newspaper   
67218             3                           Marsh averts cash crunch 

### 2.2 - Training Naïve Bayes

Create a bag of words representation for the data. Also report the number of unique words in your vocabulary, as well as in each class.

In [8]:
# code here
def vocab(df): 
    stringSplit = df['info'].str.split(" ")
    tokens = stringSplit.tolist()
    allTokens = [item for sublist in tokens for item in sublist]
    allTokens = np.array(allTokens)
    vocab = np.unique(allTokens, return_counts = True)
    return vocab

print(vocab(trainCleaned))
    

(array(['', 'a', 'aa', ..., 'zz', 'zzz', 'zzzzzz'], dtype='<U122'), array([205254, 107867,     64, ...,      3,      1,      1]))


In [9]:
vocabulary = vocab(trainCleaned)
unique_words = vocabulary[0]
print("No of unique words in vocabulary: ", len(unique_words))


No of unique words in vocabulary:  91338


In [10]:
print("No of unique words in each class- \n")
print("Class Index = 1: ", len(vocab(trainCleaned[trainCleaned['Class Index'] == 1])[0] )) 
print("Class Index = 2: ", len(vocab(trainCleaned[trainCleaned['Class Index'] == 2])[0])) 
print("Class Index = 3: ", len(vocab(trainCleaned[trainCleaned['Class Index'] == 3])[0])) 
print("Class Index = 4: ", len(vocab(trainCleaned[trainCleaned['Class Index'] == 4])[0])) 

No of unique words in each class- 

Class Index = 1:  36698
Class Index = 2:  35354
Class Index = 3:  35813
Class Index = 4:  43406


Implement a general `trainNaiveBayes` function that returns log priors and liklehoods for each class in a given dataset. Apply Laplace smoothing.

In [11]:
# code here
def trainNaiveBayes(data, C):
    
    final = []
    priors = []
    v = vocab(data)[0]
    for c in C:
        likeli = []
        N_doc = len(data)
        N_c = len(data[data["Class Index"] == c]) #how many times does class c occur in dataset.
        log_prior = math.log( N_c / N_doc,10)
        priors.append(log_prior)
        classIndex= data[data["Class Index"] == c]
        big_doc= [d for d in classIndex['info'] ]
#         big_doc= [d for d in data[data["Class Index"] == c] ]
#         print(big_doc)
        classData = data[data["Class Index"] == c] #.str.split(" ") #split into words. 
        classData = classData['info'].str.split(' ')
        counts = np.unique(np.array(big_doc), return_counts= True)
        for word in v: 
#             print(len(counts))
#             counts = big_doc.count(word)
            counts = 0
            for i in range(len(big_doc)):
                counts = counts + big_doc[i].count(word)
            likelihood = math.log((counts+1 )/ (len(classData) + len(v)), 10) 
            likeli.append(likelihood)

        final.append(likeli)
    return priors, final


In [12]:
# code here

c = trainCleaned['Class Index']
classes = np.unique(np.array(c))

log_prior, likelihoods = trainNaiveBayes(trainCleaned, classes)
# print(likelihoods[0])

Call the function on the training data to train your model.

In [13]:
likelihoods[1]

[1.7260227216848207,
 0.6038860194686199,
 -2.250849720329312,
 -4.481936840914135,
 -5.083996832242097,
 -4.2388987922278405,
 -5.083996832242097,
 -4.2388987922278405,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -3.5525179151998416,
 -4.782966836578116,
 -5.083996832242097,
 -3.6690234842712792,
 -5.083996832242097,
 -4.180906845250154,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -4.782966836578116,
 -3.0546130545568873,
 -4.782966836578116,
 -5.083996832242097,
 -4.782966836578116,
 -5.083996832242097,
 -5.083996832242097,
 -4.782966836578116,
 -5.083996832242097,
 -3.937868796563859,
 -1.4001396268417508,
 -2.6948307478775644,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -5.083996832242097,
 -3.4929322252155974,
 -3.592635138407824,
 -5.083996832242097,
 -4.606875577522435,
 -5.083996832242097,
 -5.083996832242097,
 -4.782966836578116,
 -5.0

### 2.3 - Testing Naïve Bayes

Create a `testNaiveBayes` function that return predicted class for each article.

In [16]:
trainCleaned.head(5)

,Class Index,info
0,3,wall st bears claw back into the black reuters...
1,3,carlyle looks toward commercial aerospace reut...
2,3,oil and economy cloud stocks outlook reuters r...
3,3,iraq halts oil exports from main southern pipe...
4,3,oil prices soar to alltime record posing new m...


In [17]:
V = vocab(trainCleaned)[0]

In [22]:
# code here
def testNaiveBayes(test_doc, log_prior,log_likelihood, C, V ):
    posterior = []
    for c in C:
        k = 0
        prior = log_prior[c-1]
        for i in range(len(test_doc)):
            word = test_doc[i]
            if word in V: 
                prior  = prior +  log_likelihood[c-1][k]
                k +=1
        posterior.append(prior)
#         print("c", c  , " " , prior)
        # print(posterior
    return C[np.argmax(posterior)]

# def calc_posterior(self, x):
#     posteriors = []
#     for i in range(self.count):
#         prior = np.log(self.prior[i]) 
#         conditional = np.sum(np.log(self.gaussian_density(i, x)))
#         posterior = prior + conditional
#         posteriors.append(posterior)
#     return self.classes[np.argmax(posteriors)]

training_data = trainCleaned['info'].tolist()
# k = testNaiveBayes(training_data[3], log_prior,likelihoods, classes,V)


# print(training_data[3])
# print(k,k2,k3,k4)


Create an `evaluation` function that returns the accuracy, f1 score, and confusion matrix for test predictions

In [23]:
def confusion (true, predicted):
  classNum = np.unique(true) 
  s = len(classNum) #s should be equal to 10. 
  confusion = np.zeros(shape=(s,s))  #since it is multiclass, we will have s * s matrix. 
  for i in range(s):
    for j in range(s): 
      confusion[i][j] = np.sum( (predicted == classNum[j])  & (true == classNum[i]))
  return confusion 


def getFalse(i, arr): #false for one specific class only. 
  false = 0
  for j in range(len(arr)):
    if (j!= i): 
      false = false + arr[j]
  return false

def FNlist(arr, i): 
  false = []
  for j in range(len(arr)):
    false.append(arr[j][i])

  return false 

def macroF1 (TP, FP, FN): 
  f1 = 0
  for i in range(len(TP)):
    f1 = f1 +  ( (2 * TP[i]) / (2*TP[i] + FP[i] + FN[i]))
  ave = f1 / len(TP)
  return ave



In [24]:
# code here

def eval(true, predicted): 
  conf = confusion (true, predicted) #returns confusion matrix. 
  #classification accuracy. 
  accuracy = []
  trueP = []
  trueN = []
  falsePos = []
  falseNeg = []
  classNum = len(np.unique(true))

  for i in range(classNum): #no of classes 
   TP = conf[i][i]  #true positive.
   total = conf.sum() 
   TN = 0
   falsePos.append(getFalse(i, conf[i]))
   falseNeg.append(getFalse(i, FNlist(conf, i)))

   #for true negative
   for j in range(classNum):
    #  print("first: i ", i, " j: ", j)
    for k in range(classNum):
        if j != i and k != i:
          if (j < j - 1):
            j = j + 1
          if (k < k - 1):
            k = k + 1  
          TN = TN + conf[j][k]
          #print("j: ", j, " k: ", k)
   a = (TP + TN) / total
   trueP.append(TP) #add true positives to true positive list.
   trueN.append(TN) #add true negatives to true negative list.

   accuracy.append((TP + TN)/total)  

  averageAccuracy = sum(accuracy)/ len(accuracy) #find average accuracy of all classes. 
  mF1 = macroF1(trueP, falsePos, falseNeg)
  total = [conf, averageAccuracy, mF1]

  return total 


Use the defined functions to predict labels for the test data and evaluate your model's performance. Print the output.

In [ ]:
# code here
testData = test['info'].tolist()
test_output = test['Class Index'].tolist()
pred  = []
for i in range(len(testData)):
    class_prediction = testNaiveBayes(testData[i], log_prior,likelihoods, classes,V)
    pred.append(class_prediction)


In [31]:
performance = eval(np.array(test_output), np.array(pred) )
print(performance)

[array([[261.,   0.,   1.,   0.],
       [268.,   0.,   0.,   0.],
       [202.,   0.,   1.,   0.],
       [245.,   0.,   0.,   0.]]), 0.6339468302658486, 0.10785097915599512]


### 2.4 - Discussion

Answer the following questions:
- How did you decide what kind of pre-processing to do? How did this effect the performance of the classifier?
- How does your f1 score rate the performance of your model? What can be done to improve it?
- The given dataset has an equal number of samples in each class. How, if at all, would the performance of the model change if the data had a class imbalance?
- The fundamental Naïve Bayes assumption is that each feature makes an independent and equal contribution to the outcome. Is this strictly true for this dataset? How might the assumption have impacted the predictions?

Answer here:

Q1: I decided to make all words lower case, remove puncutations, numbers and stop words. This decreased the size of the vocabulary. It also improves the performaance of the classifier. 



Q2: f1 scores measures the classifier's performance. It usually is used to measure performance of imbalanced data.  



Q3: For imbalanced classes the accuracy of the model would be very less. However, the F1 score might not be less since it can work with imbalanced data. 


Q4: This is not strictly true for the dataset. If we had not made the assumption, the probability would reduce greatly since it would be very difficult to find dependent features. 
